<a href="https://colab.research.google.com/github/cloveryo/blogImage/blob/main/assignment01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# DOWNLOAD DATA

In [2]:
!wget https://github.com/JovianML/opendatasets/raw/master/data/house-prices-advanced-regression-techniques.zip
!unzip house-prices-advanced-regression-techniques.zip

--2022-10-07 14:19:26--  https://github.com/JovianML/opendatasets/raw/master/data/house-prices-advanced-regression-techniques.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JovianML/opendatasets/master/data/house-prices-advanced-regression-techniques.zip [following]
--2022-10-07 14:19:26--  https://raw.githubusercontent.com/JovianML/opendatasets/master/data/house-prices-advanced-regression-techniques.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203809 (199K) [application/zip]
Saving to: ‘house-prices-advanced-regression-techniques.zip’

house-prices-advanc 100%[===================>] 

In [12]:
# contact dataset
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(len(train), len(test))
dataset = pd.concat([train, test], axis=0)
print(dataset.shape)
dataset = dataset.reset_index(drop=True)

1460 1459
(2919, 81)


In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2919 non-null   object 
 3   LotFrontage    2919 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Utilities      2919 non-null   object 
 6   LotConfig      2919 non-null   object 
 7   LandSlope      2919 non-null   object 
 8   Neighborhood   2919 non-null   object 
 9   Condition1     2919 non-null   object 
 10  Condition2     2919 non-null   object 
 11  BldgType       2919 non-null   object 
 12  HouseStyle     2919 non-null   object 
 13  OverallQual    2919 non-null   int64  
 14  OverallCond    2919 non-null   int64  
 15  YearBuilt      2919 non-null   int64  
 16  YearRemodAdd   2919 non-null   int64  
 17  RoofStyle      2919 non-null   object 
 18  RoofMatl

# PREPROCESSING

In [ ]:
# Handle NA
# string features
columns = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']
for col in columns:
    dataset[col].fillna(f'None', inplace=True)

# numerical features
columns = ['LotFrontage', 'MasVnrArea', 'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea']
for col in columns:
    dataset[col].fillna(0, inplace=True)

# drop columns that hard to be handled
columns = ['Street', 'Alley', 'LotShape', 'LandContour', 'GarageYrBlt', 'YearBuilt', 'YearRemodAdd', 'MoSold', 'YrSold'] # 删掉日期类feature和一些看起来没用的feature
for col in columns:
    dataset.drop(col, axis=1,  inplace=True)

# reindex
dataset = dataset.reset_index(drop=True)
print(dataset.shape)
dataset.info()

In [ ]:
# One-Hot
columns = ['MSSubClass', 'OverallQual', 'OverallCond']
for col in columns:
    dataset[col] = dataset[col].astype(str)
dataset = pd.get_dummies(dataset)
print(dataset.shape)

# PREPARE DATASET

In [16]:
gt = dataset.SalePrice
gt = gt.dropna(how='any')
gt = gt.reset_index(drop=True)
dataset = dataset.drop(['SalePrice'], axis=1)

In [17]:
# split train test
train = dataset[dataset['Id'].isin(train['Id'])]
train = train.drop(['Id'], axis=1)
train = train.reset_index(drop=True)

test = dataset[dataset['Id'].isin(test['Id'])]
test = test.reset_index(drop=True)
test_id = test.Id
test = test.drop(['Id'], axis=1)

# TRAIN A LINEAR REGRESSION MODEL

In [18]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model = model.fit(train, gt)

# EVALUATE MODEL

In [19]:
r_sq = model.score(train, gt)
print(f'score: {r_sq}')

score: 0.9999960767432823


# MAKE PREDICTIONS

In [20]:
y_pred = model.predict(test)
output = pd.concat([test_id, pd.DataFrame(y_pred)], axis=1)
output.to_csv('output.csv', index=False)
output

,Id,0
0,1461,124884.461233
1,1462,55296.315452
2,1463,128182.647324
3,1464,166914.101546
4,1465,325185.176640
...,...,...
1454,2915,98277.418297
1455,2916,60454.281888
1456,2917,67175.806308
1457,2918,-7356.937477
